In [ ]:
import tensorflow as tf
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:



# 1. Define paths
model_path = "/content/drive/MyDrive/Potato model/Model.h5"  # Your .h5 model path
model_dir = os.path.dirname(model_path)  # Get directory of .h5 file

In [ ]:
output_name = os.path.splitext(os.path.basename(model_path))[0] + "_quant.tflite"  # New filename
tflite_path = os.path.join(model_dir, output_name)  # Full output path

# 2. Load the model
try:
    model = tf.keras.models.load_model(model_path)
    print(" Model loaded successfully")
    print(f"Input shape: {model.input_shape}")  # Verify expected input shape
except Exception as e:
    print(f" Failed to load model: {e}")
    raise

# 3. Representative dataset (adjust shape if different from 224x224x3)
def representative_dataset():
    input_shape = model.input_shape[1:]  # Get (224, 224, 3) from (None, 224, 224, 3)
    for _ in range(100):  # 100 representative samples
        yield [np.random.rand(1, *input_shape).astype(np.float32)]

# 4. Configure converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
    tf.lite.OpsSet.SELECT_TF_OPS  # Fallback for unsupported ops
]

# 5. Convert and save
try:
    tflite_model = converter.convert()
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print(f"Quantization successful! Model saved to: {tflite_path}")
    print(f"File size: {os.path.getsize(tflite_path)/1024:.1f} KB")
except Exception as e:
    print(f"Conversion failed: {e}")
    if "representative_dataset" in str(e):
        print("Tip: Check if your representative dataset matches the model's input shape")
    raise

✅ Model loaded successfully
Input shape: (None, 224, 224, 3)
Saved artifact at '/tmp/tmp_ohfe4vu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  132550197277328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197275600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197276176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197274640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197275216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197273680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197274256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197272912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197272720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132550197278672: TensorS

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Quantization successful! Model saved to: /content/drive/MyDrive/Potato model/Model_quant.tflite
File size: 6412.8 KB
